In [1]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [2]:
cancer_model = BayesianModel([('Pollution','Cancer'),('Smoker','Cancer'),('Cancer','Xray'),('Cancer','Dyspnoea')])
print("Bayesian network nodes are ",cancer_model.nodes())
print("Bayesian network edges are ",cancer_model.edges())

Bayesian network nodes are  ['Pollution', 'Cancer', 'Smoker', 'Xray', 'Dyspnoea']
Bayesian network edges are  [('Pollution', 'Cancer'), ('Cancer', 'Xray'), ('Cancer', 'Dyspnoea'), ('Smoker', 'Cancer')]


In [3]:
cpd_poll = TabularCPD(variable='Pollution',variable_card=2,values=[[0.9],[0.1]])
cpd_smoke = TabularCPD(variable='Smoker',variable_card=2,values=[[0.3],[0.7]])
cpd_cancer = TabularCPD(variable='Cancer',variable_card=2,values=[[0.03,0.05,0.001,0.02],[0.97,0.95,0.999,0.98]],evidence=['Smoker','Pollution'],evidence_card=[2,2])
cpd_xray = TabularCPD(variable='Xray',variable_card=2,values=[[0.9,0.2],[0.1,0.8]],evidence=['Cancer'],evidence_card=[2])
cpd_dysp = TabularCPD(variable='Dyspnoea',variable_card=2,values=[[0.65,0.3],[0.35,0.7]],evidence=['Cancer'],evidence_card=[2])

In [4]:
cancer_model.add_cpds(cpd_poll,cpd_smoke,cpd_cancer,cpd_xray,cpd_dysp)

In [5]:
print("Checking correctness")
print(cancer_model.check_model())

Checking correctness
True


In [6]:
print("All local independencies\n")
cancer_model.get_independencies()

All local independencies



(Pollution ⟂ Smoker)
(Pollution ⟂ Xray, Dyspnoea | Cancer)
(Pollution ⟂ Xray, Dyspnoea | Smoker, Cancer)
(Pollution ⟂ Dyspnoea | Xray, Cancer)
(Pollution ⟂ Xray | Cancer, Dyspnoea)
(Pollution ⟂ Dyspnoea | Smoker, Xray, Cancer)
(Pollution ⟂ Xray | Smoker, Cancer, Dyspnoea)
(Smoker ⟂ Pollution)
(Smoker ⟂ Xray, Dyspnoea | Cancer)
(Smoker ⟂ Dyspnoea | Xray, Cancer)
(Smoker ⟂ Xray, Dyspnoea | Pollution, Cancer)
(Smoker ⟂ Xray | Cancer, Dyspnoea)
(Smoker ⟂ Dyspnoea | Xray, Pollution, Cancer)
(Smoker ⟂ Xray | Pollution, Cancer, Dyspnoea)
(Xray ⟂ Smoker, Pollution, Dyspnoea | Cancer)
(Xray ⟂ Pollution, Dyspnoea | Smoker, Cancer)
(Xray ⟂ Smoker, Dyspnoea | Pollution, Cancer)
(Xray ⟂ Smoker, Pollution | Cancer, Dyspnoea)
(Xray ⟂ Dyspnoea | Smoker, Pollution, Cancer)
(Xray ⟂ Pollution | Smoker, Cancer, Dyspnoea)
(Xray ⟂ Smoker | Pollution, Cancer, Dyspnoea)
(Dyspnoea ⟂ Smoker, Xray, Pollution | Cancer)
(Dyspnoea ⟂ Xray, Pollution | Smoker, Cancer)
(Dyspnoea ⟂ Smoker, Pollution | Xray, Cancer)
(Dy

In [7]:
cancer_infer = VariableElimination(cancer_model)

In [8]:
print("Probability of Cancer given smoker")
q = cancer_infer.query(variables=['Cancer'],evidence={'Smoker': 1})
print(q)

Eliminating: Dyspnoea: 100%|██████████| 3/3 [00:00<00:00, 499.64it/s]Probability of Cancer given smoker
+-----------+---------------+
| Cancer    |   phi(Cancer) |
+===========+===============+
| Cancer(0) |        0.0029 |
+-----------+---------------+
| Cancer(1) |        0.9971 |
+-----------+---------------+



In [9]:
print("Probability of Cancer given smoker and polluion")
q = cancer_infer.query(variables=['Cancer'],evidence={'Smoker': 1,'Pollution': 1})
print(q)

Probability of Cancer given smoker and polluion
Eliminating: Dyspnoea: 100%|██████████| 2/2 [00:00<00:00, 669.54it/s]+-----------+---------------+
| Cancer    |   phi(Cancer) |
+===========+===============+
| Cancer(0) |        0.0200 |
+-----------+---------------+
| Cancer(1) |        0.9800 |
+-----------+---------------+

